In [1]:
#importing packages
import bs4
from selenium import webdriver
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
import requests
import re
import pandas as pd
import time
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

In [2]:
#setting up selenium driver
my_url = 'https://www.kickstarter.com/discover/advanced?ref=discovery_overlay&sort=distance'
driver = webdriver.Chrome()
driver.maximize_window()
driver.get(my_url)
driver.implicitly_wait(10)
time.sleep(2)

In [3]:
element = driver.find_element_by_xpath('//*[@id="projects"]/div[4]/a')
driver.execute_script("arguments[0].click();", element)
element.send_keys(Keys.END)
#time.sleep(30)
element.send_keys(Keys.HOME)

In [4]:
links = driver.find_elements_by_css_selector('.soft-black.mb3')

In [5]:
len(links)

47

In [6]:
pages = []
for link in links:
    pages.append(link.get_attribute('href'))
len(pages)
driver.quit()

In [12]:
titles = []
for page in pages:
    uClient = uReq(page)
    page_html = uClient.read()
    uClient.close()
    page_soup = soup(page_html,"html.parser")
    
    title = page_soup.find("h2",{"type-28 type-24-md soft-black mb1 project-name"}).text
    pledge = page_soup.findAll("div",{"class":"pledge__info"})
    out_path = "D:\\Proneeth\\Kickstarter\\"
    title = title.replace(":","").replace(" ","").replace("/","").replace("\\","").replace('"','').replace('*','').replace('?','').replace('<','').replace('>','').replace('|','')
    title = out_path + title + '.xlsx'
    
    writer = pd.ExcelWriter(title , engine='xlsxwriter')
    #looping through all the pledge infos and saving it into a dataframe
    headers = ["Pledge_Amount", "Estimated_Delivery_Time","Backer_Count"]
    df = pd.DataFrame(columns = headers)

    if pledge[0].find("h2",{"class":"pledge__amount"}).text == '\nPledge without a reward\n':
        for info in pledge[1:len(pledge)]:
            pledge_amount = info.find("span","pledge__currency-conversion").span.text
            estimated_time_info = info.find("time","invisible-if-js js-adjust-time").text
            pledge_backer_count = re.search("\d+",info.find("span","pledge__backer-count").text).group(0)
            row = pd.Series([pledge_amount,estimated_time_info,pledge_backer_count],index = headers)
            df = df.append(row,ignore_index = True)

    else:
        for info in pledge:
            pledge_amount = info.find("span","pledge__currency-conversion").span.text
            estimated_time_info = info.find("time","invisible-if-js js-adjust-time").text
            pledge_backer_count = re.search("\d+",info.find("span","pledge__backer-count").text).group(0)
            row = pd.Series([pledge_amount,estimated_time_info,pledge_backer_count],index = headers)
            df = df.append(row,ignore_index = True)
    
    df.to_excel(writer, sheet_name='Pledge Info')
    
    #grabbing the href for the faq page
    faq_page_link = page_soup.find("a",{"class":"js-load-project-content js-load-project-faqs mx3 project-nav__link--faqs tabbed-nav__link type-14"})["href"]
    faq_page_link = "https://www.kickstarter.com" + faq_page_link
    
    #setting up selenium driver
    driver = webdriver.Chrome()
    driver.get(faq_page_link)
    driver.implicitly_wait(10)
    time.sleep(3)
    
    #Clicking on the faq buttons
    from selenium.common.exceptions import NoSuchElementException

    elements = driver.find_elements_by_css_selector('.js-faq-question-toggle.block.p3.no-outline.flex.items-center')
    try:
        for element in elements:
            driver.execute_script("arguments[0].click();", element)
        html_faq = driver.page_source
        page_soup_faq = soup(html_faq,"html.parser")
    except NoSuchElementException:
        html_faq = driver.page_source
        page_soup_faq = soup(html_faq,"html.parser")
        
    #making a faq dataframe
    faq_headers = ["Question","Answer"]
    df_faq = pd.DataFrame(columns = faq_headers)
    elements = page_soup_faq.findAll("li",{"class":"js-faq bg-white border mb2 shadow-button radius2px hover-bg-grey-200 js-expanded"})

    for element in elements:
        question = element.find("a",{"class":"js-faq-question-toggle block p3 no-outline flex items-center"}).text
        answer =   element.find("div",{"class":"type-14 navy-700 normal"}).text  
        row = pd.Series([question,answer],index = faq_headers)
        df_faq = df_faq.append(row,ignore_index = True)
            
    df_faq.to_excel(writer, sheet_name='FAQs')
    
    #clicking on updates button
    element_updates = driver.find_element_by_xpath('//*[@id="updates-emoji"]')
    driver.execute_script("arguments[0].click();", element_updates)
    driver.implicitly_wait(10)
    time.sleep(3)
    
    
    #Checking for the see more button

    try:
        element = driver.find_element_by_xpath('//*[@id="project-post-interface"]/div/div[11]/div/div/div/button')
        driver.execute_script("arguments[0].click();", element)
        driver.implicitly_wait(10)
        time.sleep(3)
        html_update = driver.page_source
        page_soup_updates = soup(html_update,"html.parser")
    except NoSuchElementException:
        html_update = driver.page_source
        page_soup_updates = soup(html_update,"html.parser") 
        
        
    #checking the number of updates
    updates = page_soup_updates.findAll("a",{"class":"truncated-post soft-black block border border-grey-500 hover-border-dark-grey-400"})
    number_of_updates = len(updates)
    
    #checking the number of updates visible only for backers
    backer_updates = page_soup_updates.findAll("div",{"class":"truncated-post soft-black block border border-grey-500 hover-border-dark-grey-400"})
    number_of_backer_updates = len(backer_updates)
    
    
    #making a dataframe for the updates information
    updates_headers = ["Number","Title","Date","Number_of_comments","Number_of_likes"]
    df_updates = pd.DataFrame(columns = updates_headers)

    for update in updates:
        update_number = update.find("span",{"class":"type-13 soft-black_50 text-uppercase"}).text
        update_title = update.find("h2",{"class":"mb3"}).text
        update_date = update.find("span",{"class":"type-13 soft-black_50 block-md"}).text

        if not update.findAll("span",{"class":"mr4"})[0].text:
            comments = 0
        else:    
            comments = update.findAll("span",{"class":"mr4"})[0].text

        if not update.findAll("span",{"class":"mr4"})[1].text:
            likes = 0
        else:
            likes = update.findAll("span",{"class":"mr4"})[1].text

        row = pd.Series([update_number,update_title,update_date,comments,likes],index = updates_headers)
        df_updates = df_updates.append(row,ignore_index = True)

    for update in backer_updates:
        update_number = update.find("span",{"class":"type-13 soft-black_50 text-uppercase"}).text
        update_title = update.find("h2",{"class":"mb3"}).text
        update_date = update.find("span",{"class":"type-13 soft-black_50 block-md"}).text

        if not update.findAll("span",{"class":"mr4"})[0].text:
            comments = 0
        else:    
            comments = update.findAll("span",{"class":"mr4"})[0].text

        if not update.findAll("span",{"class":"mr4"})[1].text:
            likes = 0
        else:
            likes = update.findAll("span",{"class":"mr4"})[1].text

        row = pd.Series([update_number,update_title,update_date,comments,likes],index = updates_headers)
        df_updates = df_updates.append(row,ignore_index = True)
    
    df_updates.to_excel(writer, sheet_name='Updates Info')
    
    #clicking on comments tab
    element_comments = driver.find_element_by_xpath('//*[@id="comments-emoji"]')
    driver.execute_script("arguments[0].click();", element_comments)
    driver.implicitly_wait(10)
    time.sleep(3)
    
    #clicking on show more comments until it shows all
    condition = True
    while condition:
        try:
            element = driver.find_element_by_xpath('//*[@id="react-project-comments"]/div/button')
            driver.execute_script("arguments[0].click();", element)
            driver.implicitly_wait(10)
            time.sleep(5)
        except NoSuchElementException:
            condition = False 
    
    #clicking on show more replies until it shows all
    condition = True
    while condition:
        try:
            element = driver.find_element_by_css_selector('.bttn.bttn-medium.bttn-white.flex.w100p.keyboard-focusable')
            driver.execute_script("arguments[0].click();", element)
            time.sleep(5)
            driver.implicitly_wait(10)
        except NoSuchElementException:
            condition = False 

    #parsing the comments page
    html_comments = driver.page_source
    page_soup_comments = soup(html_comments,"html.parser")
    
    page_comments = page_soup_comments.findAll("div",{"class":"border-box relative break-word border border-grey-400 px3 pt3 pb2 o100p transition-all transition-delay-1000 bg-white"})
    total_comments = len(page_comments)
    
    #making a dataframe for the comments section
    comments_headers = ["UserName","Comment","Status","Time"]
    df_comments = pd.DataFrame(columns = comments_headers)

    for comment in page_comments:
        user_name = comment.find("span",{"class":"mr2"}).text
        user_comment = comment.find("div",{"class":"w100p"}).text
        if not comment.findAll("span",{"class":"apricot-600 type-14 mr1"}):
            if not comment.findAll("span",{"class":"bg-ksr-green-700 white px1 type-14 mr1"}):
                user_status = "backer"
            else:
                user_status = comment.find("span",{"class":"bg-ksr-green-700 white px1 type-14 mr1"}).text
        else:
            user_status = comment.find("span",{"class":"apricot-600 type-14 mr1"}).text 
        user_time = comment.find("time",{"class":"block dark-grey-400 type-12"})["title"]

        row = pd.Series([user_name,user_comment,user_status,user_time],index = comments_headers)
        df_comments = df_comments.append(row,ignore_index = True)
    
    df_comments.to_excel(writer, sheet_name='Comments')
    
    #clicking on community tab
    element_community = driver.find_element_by_xpath('//*[@id="community-emoji"]')
    driver.execute_script("arguments[0].click();", element_community)
    driver.implicitly_wait(10)
    time.sleep(2)
    
    
    #parsing the community page
    html_community = driver.page_source
    page_soup_community = soup(html_community,"html.parser") 
    
    #making a dataframe for the community top cities section
    cities= page_soup_community.find("div",{"class":"community-section__locations_cities"})

    if cities:
        cities_headers = ["City","Backers"]
        df_cities = pd.DataFrame(columns = cities_headers)
        cities_list = cities.findAll("div",{"class":"location-list__item js-location-item"})

        for city in cities_list:
            city_name = city.find("div",{"class":"primary-text js-location-primary-text"}).text
            backers = city.find("div",{"class":"tertiary-text js-location-tertiary-text"}).text
            row = pd.Series([city_name,backers],index = cities_headers)
            df_cities = df_cities.append(row,ignore_index = True)
        
        df_cities.to_excel(writer, sheet_name='City-wise-backers')
            
    #making a dataframe for the community top countries section
    countries = page_soup_community.find("div",{"class":"community-section__locations_countries"})

    if countries:
        country_headers = ["Country","Backers"]
        df_countries = pd.DataFrame(columns = country_headers)

        countries_list = countries.findAll("div",{"class":"location-list__item js-location-item"})
        for country in countries_list:
            country_name = country.find("div",{"class":"primary-text js-location-primary-text"}).text
            backers = country.find("div",{"class":"tertiary-text js-location-tertiary-text"}).text
            row = pd.Series([country_name,backers],index = country_headers)
            df_countries = df_countries.append(row,ignore_index = True)
        
        df_countries.to_excel(writer, sheet_name='Country-wise-backers')
    
    
    #extracting new and existing backer count details
    
    if page_soup_community.find("div",{"class":"new-backers"}):
        new_backers_count = page_soup_community.find("div",{"class":"new-backers"}).find("div",{"class":"count"}).text
        if page_soup_community.find("div",{"class":"existing-backers"}):    
            returning_backers_count = page_soup_community.find("div",{"class":"existing-backers"}).find("div",{"class":"count"}).text
        data = {'New Backers':[new_backers_count],'Returning Backers':[returning_backers_count]}
        df_new_and_old = pd.DataFrame(data,columns = ['New Backers','Returning Backers'])
        df_new_and_old.to_excel(writer, sheet_name='New and Returning backers')
    
    driver.quit()
    writer.save()